In [1]:
import numpy as np
import pandas as pd
import requests
import csv
import ssl
from urllib.request import urlopen
from urllib.parse import quote_plus

In [161]:
pd.set_option('display.max_columns', None)

# Top_1

1. color를 원핫인코딩 변경
2. clothSexCommon이 1인 값은 남자와 여자 모두 1로 변경
3. user테이블에 그 user가 산 옷들 평균내서 fit ~ thickness ~ season까지 저장

In [187]:
top_1 = pd.read_csv('C:\\Users\\SSAFY\\Desktop\\ssafy\\DA_pjt\\S06P22E202\\data\\DataCrawling\\DATA\\Cloth\\musinsa_clothes_top_1.csv')

In [188]:
top_1.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [189]:
top_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43356 entries, 0 to 43355
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            43356 non-null  int64  
 1   largeCategory      43356 non-null  int64  
 2   largeCategoryName  43356 non-null  object 
 3   smallCategory      43356 non-null  int64  
 4   smallCategoryName  43356 non-null  object 
 5   colorNo            43356 non-null  int64  
 6   clothName          43356 non-null  object 
 7   brand              43356 non-null  object 
 8   image              43356 non-null  object 
 9   clothPrice         43356 non-null  object 
 10  date               43356 non-null  int64  
 11  hashtags           43356 non-null  object 
 12  clothSexMen        43356 non-null  int64  
 13  clothSexWomen      43356 non-null  int64  
 14  clothSexCommon     43356 non-null  int64  
 15  clothRate          43356 non-null  float64
 16  clothReviewNo      433

In [190]:
top_1.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,4.335600e+04,43356.0,43356.00000,43356.00000,4.335600e+04,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000
mean,1.777224e+06,1.0,103.68844,5.90239,2.020498e+07,0.705808,0.256228,0.037619,2.287314,1.145678,0.627110,1.054733,1.277816,0.919042,0.212566,0.153335,0.209521,0.147015
std,5.292900e+05,0.0,2.95444,8.85016,1.426893e+04,0.455684,0.436554,0.190275,2.373989,1.890802,1.106783,1.734553,2.039596,1.497869,0.409128,0.360314,0.406972,0.354125
min,1.837900e+04,1.0,101.00000,1.00000,2.012043e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.491044e+06,1.0,101.00000,1.00000,2.020062e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.927994e+06,1.0,103.00000,1.00000,2.021043e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.174280e+06,1.0,105.00000,3.00000,2.021101e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413288e+06,1.0,111.00000,25.00000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


### smallCategory 원핫인코딩 변환

In [191]:
top_1.insert(4, "smallCategory_others", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_short", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_long", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_knit", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_sweatshirt", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_hoody", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_collar", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_shirt", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_halfshort", top_1['smallCategory'], True)

In [192]:
top_1['smallCategory_halfshort'] = top_1['smallCategory_short'].apply(lambda x: 1 if x == 101 else 0)
top_1['smallCategory_shirt'] = top_1['smallCategory_shirt'].apply(lambda x: 1 if x == 102 else 0)
top_1['smallCategory_collar'] = top_1['smallCategory_collar'].apply(lambda x: 1 if x == 103 else 0)
top_1['smallCategory_hoody'] = top_1['smallCategory_hoody'].apply(lambda x: 1 if x == 104 else 0)
top_1['smallCategory_sweatshirt'] = top_1['smallCategory_sweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_1['smallCategory_knit'] = top_1['smallCategory_knit'].apply(lambda x: 1 if x == 106 else 0)
top_1['smallCategory_long'] = top_1['smallCategory_long'].apply(lambda x: 1 if x == 110 else 0)
top_1['smallCategory_short'] = top_1['smallCategory_short'].apply(lambda x: 1 if x == 111 else 0)
top_1['smallCategory_others'] = top_1['smallCategory_others'].apply(lambda x: 1 if x == 108 else 0)

In [193]:
top_1 = top_1.drop('smallCategory', axis=1)

In [194]:
top_1.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothSexCommon',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [196]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top_1[top_1.columns[i]].value_counts()[1]
    print(top_1[top_1.columns[i]].value_counts())
print(num)
print(top_1.shape)

0    26680
1    16676
Name: smallCategory_halfshort, dtype: int64
0    39437
1     3919
Name: smallCategory_shirt, dtype: int64
0    42115
1     1241
Name: smallCategory_collar, dtype: int64
0    38206
1     5150
Name: smallCategory_hoody, dtype: int64
0    34453
1     8903
Name: smallCategory_sweatshirt, dtype: int64
0    41312
1     2044
Name: smallCategory_knit, dtype: int64
0    39215
1     4141
Name: smallCategory_long, dtype: int64
0    42478
1      878
Name: smallCategory_short, dtype: int64
0    42952
1      404
Name: smallCategory_others, dtype: int64
43356
(43356, 34)


### clothSexCommon 변환

In [197]:
for i in top_1['clothSexCommon']:
    if i == 1:
        top_1['clothSexMen'] = 1
        top_1['clothSexWomen'] = 1

In [198]:
top_1 = top_1.drop('clothSexCommon', axis=1)

In [199]:
top_1.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

In [204]:
print(top_1['clothSexMen'].value_counts())

1    43356
Name: clothSexMen, dtype: int64


### color 원핫인코딩 변환

#### 원피스
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 스커트
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 바지
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 아우터
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [200]:
top_1.insert(14, "color_others", top_1['colorNo'], True)
top_1.insert(14, "color_pattern", top_1['colorNo'], True)
top_1.insert(14, "color_jean", top_1['colorNo'], True)
top_1.insert(14, "color_beige", top_1['colorNo'], True)
top_1.insert(14, "color_brown", top_1['colorNo'], True)
top_1.insert(14, "color_purple", top_1['colorNo'], True)
top_1.insert(14, "color_blue", top_1['colorNo'], True)
top_1.insert(14, "color_green", top_1['colorNo'], True)
top_1.insert(14, "color_yellow", top_1['colorNo'], True)
top_1.insert(14, "color_ivory", top_1['colorNo'], True)
top_1.insert(14, "color_orange", top_1['colorNo'], True)
top_1.insert(14, "color_pink", top_1['colorNo'], True)
top_1.insert(14, "color_red", top_1['colorNo'], True)
top_1.insert(14, "color_black", top_1['colorNo'], True)
top_1.insert(14, "color_grey", top_1['colorNo'], True)
top_1.insert(14, "color_white", top_1['colorNo'], True)

In [201]:
top_1['color_white'] = top_1['color_white'].apply(lambda x: 1 if x == 1 else 0)
top_1['color_grey'] = top_1['color_grey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_1['color_black'] = top_1['color_black'].apply(lambda x: 1 if x == 2 else 0)
top_1['color_red'] = top_1['color_red'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_1['color_pink'] = top_1['color_pink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_1['color_orange'] = top_1['color_orange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_1['color_ivory'] = top_1['color_ivory'].apply(lambda x: 1 if x == 23 else 0)
top_1['color_yellow'] = top_1['color_yellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_1['color_green'] = top_1['color_green'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_1['color_blue'] = top_1['color_blue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_1['color_purple'] = top_1['color_purple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_1['color_brown'] = top_1['color_brown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_1['color_beige'] = top_1['color_beige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_1['color_jean'] = top_1['color_jean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_1['color_pattern'] = top_1['color_pattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_1['color_others'] = top_1['color_others'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [157]:
print(top_1['color_grey'].describe())

count    43356.000000
mean         0.476774
std          0.499466
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: color_grey, dtype: float64


In [202]:
print(top_1['color_white'].value_counts())
print(top_1['color_grey'].value_counts())

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64


In [182]:
print(top_1.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'color_white', 'color_grey', 'color_black', 'color_red',
       'color_pink', 'color_orange', 'color_ivory', 'color_yellow',
       'color_green', 'color_blue', 'color_purple', 'color_brown',
       'color_beige', 'color_jean', 'color_pattern', 'color_others',
       'clothName', 'brand', 'image', 'clothPrice', 'date', 'hashtags',
       'clothSexMen', 'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')


In [185]:
# color도 모든 1의 값의 개수 == 행의 개수
for i in range(14, 30):
    print(top_1[top_1.columns[i]].value_counts())
print(top_1.shape)

1    22685
0    20671
Name: color_white, dtype: int64
0    22685
1    20671
Name: color_grey, dtype: int64
0    43356
Name: color_black, dtype: int64
0    43356
Name: color_red, dtype: int64
0    43356
Name: color_pink, dtype: int64
0    43356
Name: color_orange, dtype: int64
0    43356
Name: color_ivory, dtype: int64
0    43356
Name: color_yellow, dtype: int64
0    43356
Name: color_green, dtype: int64
0    43356
Name: color_blue, dtype: int64
0    43356
Name: color_purple, dtype: int64
0    43356
Name: color_brown, dtype: int64
0    43356
Name: color_beige, dtype: int64
0    43356
Name: color_jean, dtype: int64
0    43356
Name: color_pattern, dtype: int64
0    43356
Name: color_others, dtype: int64
0
(43356, 49)


In [165]:
top_1.head(30)

,clothId,largeCategory,largeCategoryName,smallCategory_halfshort,smallCategory_shirt,smallCategory_collar,smallCategory_hoody,smallCategory_sweatshirt,smallCategory_knit,smallCategory_long,smallCategory_short,smallCategory_others,smallCategoryName,colorNo,color_white,color_grey,color_black,color_red,color_pink,color_orange,color_ivory,color_yellow,color_green,color_blue,color_purple,color_brown,color_beige,color_jean,color_pattern,color_others,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
0,1929966,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(153),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
1,1927658,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,EMBLEM SLOGAN LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
2,1927660,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CUTTING PATCH LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,1,4.5,2,0,0,0,0,0,0,0,0,0
3,1928209,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CLUB ARCH LOGO TEE (WHITE),스텝온리,//image.msscdn.net/images/goods_img/20210429/1...,"26,600원",20210429,[],1,1,5.0,1,0,0,0,0,0,0,0,0,0
4,1927270,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Q221-3521-1WT 남성 라운드티셔츠,프로월드컵,//image.msscdn.net/images/goods_img/20210429/1...,"24,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
5,1927634,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HEART LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['로고티셔츠', '반팔티셔츠', '그래픽티셔츠', '티셔츠', '반팔', '오와이']",1,1,5.0,6,0,0,0,0,0,0,0,0,0
6,1927575,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ROUND SLOGAN T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"39,000원",20210429,"['여름반팔티', '반팔티', '반팔티셔츠', '반팔', '그래픽티셔츠', '티셔츠...",1,1,0.0,0,0,0,0,0,0,0,0,0,0
7,1929934,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,화이트티셔츠(131),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,1,0.0,0,0,0,0,0,0,0,0,0,0
8,1927311,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,바운스 오버핏 반팔티 (화이트),어반플레이어스,//image.msscdn.net/images/goods_img/20210429/1...,"19,000원",20210429,"['오버핏', '오버핏티셔츠', '그래픽티셔츠', '루즈핏티셔츠', '티셔츠', '...",1,1,5.0,4,0,0,0,0,0,0,0,0,0
9,1927599,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BLUR WAVE LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"42,000원",20210429,"['티셔츠', '여름반팔티', '반팔티셔츠', '그래픽티셔츠', '로고티셔츠', '...",1,1,0.0,0,0,0,0,0,0,0,0,0,0




- Price에서 ',' 랑 '원' 제거
- 성별도 체크

# Top_1

1. color를 원핫인코딩 변경
2. clothSexCommon이 1인 값은 남자와 여자 모두 1로 변경
3. user테이블에 그 user가 산 옷들 평균내서 fit ~ thickness ~ season까지 저장

In [144]:
top_1 = pd.read_csv('C:\\Users\\SSAFY\\Desktop\\ssafy\\DA_pjt\\S06P22E202\\data\\DataCrawling\\DATA\\Cloth\\musinsa_clothes_top_1.csv')

In [145]:
top_1.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [146]:
top_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43356 entries, 0 to 43355
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            43356 non-null  int64  
 1   largeCategory      43356 non-null  int64  
 2   largeCategoryName  43356 non-null  object 
 3   smallCategory      43356 non-null  int64  
 4   smallCategoryName  43356 non-null  object 
 5   colorNo            43356 non-null  int64  
 6   clothName          43356 non-null  object 
 7   brand              43356 non-null  object 
 8   image              43356 non-null  object 
 9   clothPrice         43356 non-null  object 
 10  date               43356 non-null  int64  
 11  hashtags           43356 non-null  object 
 12  clothSexMen        43356 non-null  int64  
 13  clothSexWomen      43356 non-null  int64  
 14  clothSexCommon     43356 non-null  int64  
 15  clothRate          43356 non-null  float64
 16  clothReviewNo      433

In [147]:
top_1.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,4.335600e+04,43356.0,43356.00000,43356.00000,4.335600e+04,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000
mean,1.777224e+06,1.0,103.68844,5.90239,2.020498e+07,0.705808,0.256228,0.037619,2.287314,1.145678,0.627110,1.054733,1.277816,0.919042,0.212566,0.153335,0.209521,0.147015
std,5.292900e+05,0.0,2.95444,8.85016,1.426893e+04,0.455684,0.436554,0.190275,2.373989,1.890802,1.106783,1.734553,2.039596,1.497869,0.409128,0.360314,0.406972,0.354125
min,1.837900e+04,1.0,101.00000,1.00000,2.012043e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.491044e+06,1.0,101.00000,1.00000,2.020062e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.927994e+06,1.0,103.00000,1.00000,2.021043e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.174280e+06,1.0,105.00000,3.00000,2.021101e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413288e+06,1.0,111.00000,25.00000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


### smallCategory 원핫인코딩 변환

In [148]:
top_1.insert(4, "smallCategory_others", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_short", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_long", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_knit", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_sweatshirt", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_hoody", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_collar", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_shirt", top_1['smallCategory'], True)
top_1.insert(4, "smallCategory_halfshort", top_1['smallCategory'], True)

In [149]:
top_1['smallCategory_halfshort'] = top_1['smallCategory_short'].apply(lambda x: 1 if x == 101 else 0)
top_1['smallCategory_shirt'] = top_1['smallCategory_shirt'].apply(lambda x: 1 if x == 102 else 0)
top_1['smallCategory_collar'] = top_1['smallCategory_collar'].apply(lambda x: 1 if x == 103 else 0)
top_1['smallCategory_hoody'] = top_1['smallCategory_hoody'].apply(lambda x: 1 if x == 104 else 0)
top_1['smallCategory_sweatshirt'] = top_1['smallCategory_sweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_1['smallCategory_knit'] = top_1['smallCategory_knit'].apply(lambda x: 1 if x == 106 else 0)
top_1['smallCategory_long'] = top_1['smallCategory_long'].apply(lambda x: 1 if x == 110 else 0)
top_1['smallCategory_short'] = top_1['smallCategory_short'].apply(lambda x: 1 if x == 111 else 0)
top_1['smallCategory_others'] = top_1['smallCategory_others'].apply(lambda x: 1 if x == 108 else 0)

In [150]:
top_1 = top_1.drop('smallCategory', axis=1)

In [151]:
top_1.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothSexCommon',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

### clothSexCommon 변환

In [152]:
for i in top_1['clothSexCommon']:
    if i == 1:
        top_1['clothSexMen'] = 1
        top_1['clothSexWomen'] = 1

In [153]:
top_1 = top_1.drop('clothSexCommon', axis=1)

In [154]:
top_1.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategory_halfshort', 'smallCategory_shirt',
       'smallCategory_collar', 'smallCategory_hoody',
       'smallCategory_sweatshirt', 'smallCategory_knit', 'smallCategory_long',
       'smallCategory_short', 'smallCategory_others', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

### color 원핫인코딩 변환

#### 원피스
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 스커트
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 바지
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 아우터
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [155]:
top_1.insert(14, "color_others", top_1['colorNo'], True)
top_1.insert(14, "color_pattern", top_1['colorNo'], True)
top_1.insert(14, "color_jean", top_1['colorNo'], True)
top_1.insert(14, "color_beige", top_1['colorNo'], True)
top_1.insert(14, "color_brown", top_1['colorNo'], True)
top_1.insert(14, "color_purple", top_1['colorNo'], True)
top_1.insert(14, "color_blue", top_1['colorNo'], True)
top_1.insert(14, "color_green", top_1['colorNo'], True)
top_1.insert(14, "color_yellow", top_1['colorNo'], True)
top_1.insert(14, "color_ivory", top_1['colorNo'], True)
top_1.insert(14, "color_orange", top_1['colorNo'], True)
top_1.insert(14, "color_pink", top_1['colorNo'], True)
top_1.insert(14, "color_red", top_1['colorNo'], True)
top_1.insert(14, "color_black", top_1['colorNo'], True)
top_1.insert(14, "color_grey", top_1['colorNo'], True)
top_1.insert(14, "color_white", top_1['colorNo'], True)

In [156]:
top_1['color_white'] = top_1['color_white'].apply(lambda x: 1 if x == 1 else 0)
top_1['color_grey'] = top_1['color_grey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_1['color_black'] = top_1['color_black'].apply(lambda x: 1 if x == 2 else 0)
top_1['color_red'] = top_1['color_red'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_1['color_pink'] = top_1['color_pink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_1['color_orange'] = top_1['color_orange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_1['color_ivory'] = top_1['color_ivory'].apply(lambda x: 1 if x == 23 else 0)
top_1['color_yellow'] = top_1['color_yellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_1['color_green'] = top_1['color_green'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_1['color_blue'] = top_1['color_blue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_1['color_purple'] = top_1['color_purple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_1['color_brown'] = top_1['color_brown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_1['color_beige'] = top_1['color_beige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_1['color_jean'] = top_1['color_jean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_1['color_pattern'] = top_1['color_pattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_1['color_others'] = top_1['color_others'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [157]:
print(top_1['color_grey'].describe())

count    43356.000000
mean         0.476774
std          0.499466
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: color_grey, dtype: float64


In [159]:
print(top_1['color_grey'].value_counts())

0    22685
1    20671
Name: color_grey, dtype: int64
